In [157]:
import pandas as pd
from datetime import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [158]:
df = pd.read_csv('OnlineRetail.csv', delimiter=';')

In [159]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,"2,55",17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,"2,75",17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  object 
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


In [66]:
df.describe()

,Quantity,CustomerID
count,541909.000000,406829.000000
mean,9.552250,15287.690570
std,218.081158,1713.600303
min,-80995.000000,12346.000000
25%,1.000000,13953.000000
50%,3.000000,15152.000000
75%,10.000000,16791.000000
max,80995.000000,18287.000000


In [161]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [162]:
df.shape

(536641, 8)

In [163]:
df['UnitPrice'] = df['UnitPrice'].str.replace(',', '.').astype(float)

In [164]:
df = df.dropna()
df=df[df['Quantity'] > 0]
df=df[df['UnitPrice'] > 0]


In [165]:
df_aus = (df[df['Country'] =="Japan"]
 .groupby(['InvoiceNo', 'Description'])['Quantity']
 .sum().unstack().reset_index()
 .set_index('InvoiceNo'))

In [166]:
def return_one(x):
    return 1

In [167]:
df_aus.head()

Description,I LOVE LONDON MINI BACKPACK,12 PENCILS TALL TUBE RED RETROSPOT,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,36 PENCILS TUBE RED RETROSPOT,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,60 CAKE CASES DOLLY GIRL DESIGN,ABC TREASURE BOOK BOX,...,VINTAGE DONKEY TAIL GAME,VINTAGE KEEPSAKE BOX PARIS DAYS,VINTAGE LEAF MAGNETIC NOTEPAD,WALL TIDY RETROSPOT,WHITE HEART CONFETTI IN TUBE,WHITE SOAP RACK WITH 2 BOTTLES,WHITE WIRE EGG HOLDER,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN SCHOOL COLOURING SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
537218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538397,NaN,NaN,96.0,36.0,NaN,NaN,36.0,48.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,24.0,NaN,NaN
543179,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
543518,NaN,NaN,NaN,36.0,NaN,NaN,NaN,96.0,NaN,NaN,...,NaN,24.0,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
#Penyiapan data
# Tahap-1: Tiap rekord transaksi diubah ke struktur list dan disimpan di
# array list record[]
records = []
for i in range(0, 19):
    items = []
    for j in range(0, 216):
        item = str(df_aus.values[i,j])
        if(item != 'nan'):
            items.append(item)
    records.append(items)

In [171]:
# Tahap-2: Transformasi rekord-rekord list ke dataframe dg kolom = nama produk,
# nilai kolom = True atau False
# (Dibuat begitu krn library Apriori hanya menerima struktur data begitu.)  
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
store_data_transformed = pd.DataFrame(te_ary, columns=te.columns_)

In [174]:
# Analisis data untuk mendapatkan aturan asosiasi dengan memanfaatkan 
# algoritma Apriori.  
# Pada contoh ini, nilai-nilai parameter diambil cukup kecil
apriori_start = datetime.now()
from apyori import apriori
association_rules = apriori(records, min_support=0.15, min_confidence=0.6, min_lift=4, min_length=2)
association_results = list(association_rules)
apriori_end = datetime.now()

#Lihat jumlah aturan asosiasi yg dihasilkan
print(len(association_results))
print()
#Lihat aturan asosiasi pertama (ke-0)
print(association_results[0])

running_time = apriori_end-apriori_start
print("Apriori Running Time: ", str(running_time))


127

RelationRecord(items=frozenset({'36.0', '60.0', '100.0'}), support=0.15789473684210525, ordered_statistics=[OrderedStatistic(items_base=frozenset({'36.0'}), items_add=frozenset({'60.0', '100.0'}), confidence=0.75, lift=4.75), OrderedStatistic(items_base=frozenset({'60.0', '100.0'}), items_add=frozenset({'36.0'}), confidence=1.0, lift=4.75)])
Apriori Running Time:  0:00:00.146192


In [14]:
#Print semua aturan asosiasi beserta metrik-nya:
for item in association_results:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")


Rule: 120.0 -> 100.0
Support: 0.14035087719298245
Confidence: 1.0
Lift: 6.333333333333334
Rule: 144.0 -> 100.0
Support: 0.14035087719298245
Confidence: 1.0
Lift: 5.1818181818181825
Rule: 192.0 -> 100.0
Support: 0.12280701754385964
Confidence: 0.875
Lift: 6.234375
Rule: 48.0 -> 100.0
Support: 0.14035087719298245
Confidence: 1.0
Lift: 4.75
Rule: 72.0 -> 100.0
Support: 0.14035087719298245
Confidence: 1.0
Lift: 4.384615384615385
Rule: 100.0 -> 96.0
Support: 0.14035087719298245
Confidence: 1.0
Lift: 5.1818181818181825
Rule: 120.0 -> 144.0
Support: 0.14035087719298245
Confidence: 0.8888888888888888
Lift: 4.606060606060606
Rule: 120.0 -> 192.0
Support: 0.12280701754385964
Confidence: 0.7777777777777778
Lift: 5.541666666666667
Rule: 120.0 -> 48.0
Support: 0.14035087719298245
Confidence: 0.8888888888888888
Lift: 4.222222222222222
Rule: 120.0 -> 96.0
Support: 0.14035087719298245
Confidence: 0.8888888888888888
Lift: 4.606060606060606
Rule: 192.0 -> 144.0
Support: 0.12280701754385964
Confidence: 0

In [177]:
######### 
# Analisis data untuk mendapatkan aturan asosiasi dengan memanfaatkan 
# algoritma FP-Growth. 

# Buat frequent items sets dgn min support tertentu
frequent_itemsets_fpgrowth = fpgrowth(store_data_transformed, min_support=0.1, use_colnames=True)

from mlxtend.frequent_patterns import association_rules

#Buat aturan-aturan asosiasi, disaring berdasar nilai confidence tertentu
rules_conf_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence",min_threshold=0.54, support_only = False)

#Buat aturan-aturan asosiasi, disaring berdasar nilai lift tertentu
rules_lift_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="lift",min_threshold=4, support_only = False)

#Hasil analisis aturan asosiasi dengan fpgrowth (tersimpan sbg dataframe)
rules_conf = rules_conf_fpgrowth[['antecedents', 'consequents']]
rules_lift = rules_lift_fpgrowth[['antecedents', 'consequents']]

In [155]:
rules_conf

,antecedents,consequents
0,(24.0),(12.0)
1,"(24.0, 4.0)",(12.0)
2,"(4.0, 12.0)",(24.0)
3,"(24.0, 6.0)",(12.0)
4,(2.0),(12.0)
5,"(24.0, 2.0)",(12.0)
6,"(2.0, 12.0)",(24.0)
7,"(2.0, 4.0)",(12.0)
8,"(4.0, 12.0)",(2.0)
9,"(24.0, 2.0)",(6.0)


In [97]:
rules_lift

,antecedents,consequents
0,"(144.0, 72.0)",(48.0)
1,(48.0),"(144.0, 72.0)"
2,"(240.0, 72.0)",(48.0)
3,"(240.0, 48.0)",(72.0)
4,"(48.0, 72.0)",(240.0)
5,(240.0),"(48.0, 72.0)"
6,(72.0),"(240.0, 48.0)"
7,(48.0),"(240.0, 72.0)"


In [178]:

###############
# Eksperimen Perbandingan Waktu eksekusi algoritma Apriori vs FP-Growth

#Applying Apriori
apriori_start = datetime.now()
apriori_frequent_itemsets = apriori(store_data_transformed, min_support=0.12, min_confidence=0.6, min_lift=4, min_length=2, use_colnames=True)
apriori_end = datetime.now()
running_time = apriori_end-apriori_start
print("Apriori Running Time: ", str(running_time))

#Applying FP-Growth
fpgrowth_start = datetime.now()
fpgworth_frequent_itemsets = fpgrowth(store_data_transformed, min_support=0.1, use_colnames=True)
fpgrowth_end = datetime.now()
running_time = fpgrowth_end-fpgrowth_start
print("FP-Growth Running Time: ", str(running_time))

Apriori Running Time:  0:00:00
FP-Growth Running Time:  0:00:00.024205


----

In [108]:
checker = df[['StockCode', 'Description']] 

In [109]:
def covert_desc(code):
    value = checker[checker['stockcode'] == code].tail(1)
    desc = value['description'].iloc[0]
    return desc

In [110]:
def convert_to_str(list_num):
    desc = ""
    for idx, num in enumerate(list_num):   
        desc += covert_desc(num)
        if(idx != len(list_num) -1):
            desc += ", "
    return desc

In [111]:
def convert_txt(txt):
    list_comma = []
    number = []
    for idx, val in enumerate(txt):
        if(val == ","):
            list_comma.append(idx)
    print(list_comma)
    if(len(list_comma) > 0):
        number.append(txt[0:list_comma[0]])
        for coma in list_comma:
            print(coma)
            start = int(coma)+2
            finish = int(start)+5
            number.append(txt[start:finish])
        result = convert_to_str(number)    
        return result
    else:
        number.append(txt)
        result = covert_desc(number[0])
        return result

In [113]:
tulisan = []
result_aus = frequent_itemsets_fpgrowth[['antecedents', 'consequents']]

list1 = result_aus["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
list2 = result_aus["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
value_aus = pd.concat([list1, list2], axis = 1)


for index, row in value_aus.iterrows():
    print(row)
    ant = convert_txt(row['antecedents'])
    value_aus.at[index, 'antecedents'] = ant
    con = convert_txt(row['consequents'])
    value_aus.at[index, 'consequents'] = con

KeyError: "None of [Index(['antecedents', 'consequents'], dtype='object')] are in the [columns]"